# Import bibliotek i danych

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from operator import itemgetter
import time
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn import neighbors
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn import linear_model
import pickle
with open("../data/model_OLS.p", "rb") as fp:
    modelOLS = pickle.load(fp)
with open("../data/model_XGB.p", "rb") as fp:
    modelXGBoost = pickle.load(fp)
with open("../data/model_MLP.p", "rb") as fp:
    modelMLP = pickle.load(fp)
target = "stars"
SEED = 17

$\text{Wczytanie danych}$

In [2]:
data = pd.read_csv('../data/data_eda.csv')
data=data.drop(columns=['Unnamed: 0'])
data

,pages,stars,reviews,series,mix,character,plot,funny,lighthearted,emotional,...,author_stars,Fiction,Nonfiction,Literary,Fantasy,Crime,Social,Children,Romans,Realism
0,273,4.00,2017,0,0.44,0.51,0.02,0.27,0.37,0.91,...,4.305000,1,1,0,0,0,1,0,1,1
1,302,3.78,7330,0,0.39,0.42,0.17,0.03,0.01,0.18,...,3.670000,1,0,0,0,1,0,0,0,0
2,400,4.15,16761,0,0.51,0.39,0.08,0.02,0.01,0.88,...,0.000000,1,0,1,0,0,0,0,0,0
3,459,4.16,2128,1,0.48,0.10,0.40,0.04,0.02,0.07,...,0.000000,1,0,0,1,0,0,0,0,0
4,160,3.65,6634,1,0.28,0.16,0.54,0.92,0.73,0.00,...,4.115000,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6925,432,4.15,30643,0,0.48,0.05,0.46,0.00,0.00,0.40,...,3.856667,1,0,0,1,1,0,0,0,0
6926,352,3.62,1058,0,0.55,0.13,0.30,0.15,0.10,0.25,...,3.700000,1,0,0,1,0,1,0,0,0
6927,535,3.88,30975,1,0.45,0.08,0.45,0.14,0.19,0.31,...,3.870000,1,0,0,1,0,0,1,0,0
6928,472,3.88,5914,1,0.64,0.12,0.22,0.07,0.00,0.36,...,3.660000,1,0,0,1,0,0,1,0,0


$\text{Podział danych na zbiór treningowy i testowy}$

In [3]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=SEED)

$\text{Ponieważ spośród analizowanych przez nas modeli najlepsze wyniki osiągnięto dla regresji liniowej, XGBoosta oraz sieci neuronowych, do ensemblingu wykorzystamy te trzy modele.}$<p>
$\text{Tworzymy dataframe z predykcjami wszystkich modeli.}$

In [4]:
predNames = []
for result in [modelOLS, modelXGBoost, modelMLP]:
    # Tworzymy wektor predykcji tworząc serię pandasa, a następnie sortujemy indeks
    pred = pd.Series(result["predictions"], index=result["indices"]).sort_index()
    # Dodajmy kolumnę z predykcjami na podstawie nazwy z pliku
    # Wyczyścimy nazwy modeli aby zlikwidować spacje i pozbyć się niepotrzebnych składników
    cleanName = (result['name']).replace(" ", "_").split("_")[0]
    predNames.append(f"pred_{cleanName}")
    test_data[f"pred_{cleanName}"] = pred

In [5]:
test_data[predNames].corr()

,pred_OLS,pred_XGB,pred_MLP
pred_OLS,1.000000,0.878529,0.866995
pred_XGB,0.878529,1.000000,0.871951
pred_MLP,0.866995,0.871951,1.000000


$\text{Wysoka korelacja predykcji modeli.}$

In [6]:
test_data[predNames].sample(10)

,pred_OLS,pred_XGB,pred_MLP
5258,3.803262,3.761947,3.687878
5371,3.367834,3.591684,3.459280
2953,3.640516,3.618860,3.675931
3759,3.955501,3.875743,3.876574
3440,3.731564,3.575776,3.766352
5466,3.758814,3.766204,3.734923
3048,3.630619,3.598706,3.677884
2219,3.892030,3.931422,3.979347
5047,3.871370,3.837152,3.960064
6751,4.066968,4.017815,4.070896


$\text{Niewielkie różnice w predykcjach modeli.}$<p>
$\text{Sprawdźmy jakość predykcji średniej z modeli.}$

In [7]:
test_data["pred_mean"] = test_data[['pred_OLS', 'pred_XGB', 'pred_MLP']].mean(axis=1)

In [8]:
metrics.mean_squared_error(test_data['stars'], test_data['pred_mean'], squared=False)

0.1901468573628042

$\text{Jakość uśrednionych predykcji modeli wyższa niż każdego z modeli osobno.}$

In [9]:
from statsmodels.api import OLS
OLS(test_data['stars'],test_data[['pred_OLS', 'pred_XGB', 'pred_MLP']]).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  stars   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          1.985e+05
Date:                Thu, 30 May 2024   Prob (F-statistic):                        0.00
Time:                        14:12:35   Log-Likelihood:                          340.69
No. Observations:                1386   AIC:                                     -675.4
Df Residuals:                    1383   BIC:                                     -659.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
pred_OLS       0.3513      0.048      7.334      0.000       0.257       0.445
pred_XGB       0.4826      0.051      9.507      0.000       0.383       0.582
pred_MLP       0.1660      0.049      3.379      0.001       0.070       0.262
==============================================================================
Omnibus:                       37.220   Durbin-Watson:                   1.960
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               57.812
Skew:                          -0.246   Prob(JB):                     2.79e-13
Kurtosis:                       3.871   Cond. No.                         83.4
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

$\text{Przypiszemy współczynniki wag poszczególnym modelom.}$

In [11]:
test_data["pred_regg"] = 0.3513*test_data['pred_OLS'] + 0.4826*test_data['pred_XGB'] + 0.1660*test_data['pred_MLP']
metrics.mean_squared_error(test_data['stars'], test_data['pred_regg'], squared=False)

0.18923862711358064

$\text{Ostateczny wyniki modelu Ensemblingowego najwyższy spośród wszystkich modeli: 0.1894.}$